In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader 


In [2]:
#Define transforms
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5 ,0.5, 0.5], std=[0.5,0.5,0.5])
])


In [3]:
#set directories
data_dir = "D:/ds/DL/dataset/cats and dogs/training_set"
val_dir = "D:/ds/DL/dataset/cats and dogs/test_set"
train_dataset = datasets.ImageFolder(data_dir, transform=transform)
val_dataset = datasets.ImageFolder(val_dir, transform=transform)


In [4]:
#Dataloaders

train_loader = DataLoader(train_dataset, batch_size= 32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size = 32, shuffle=True)

In [5]:
#class names 
class_name = train_dataset.classes

In [6]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        # After 2 pooling layers on 128x128: (128/2/2) = 32, so shape is [64, 32, 32]
        self.fc_block = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 32 * 32, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_block(x)
        x = self.fc_block(x)
        return x


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [8]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.float().to(device).view(-1, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = (outputs > 0.5).squeeze().long()
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    print(f"Epoch {epoch+1}/{num_epochs} | Loss: {running_loss:.4f} | Val Acc: {100 * correct / total:.2f}%")


Epoch 1/10 | Loss: 163.4615 | Val Acc: 66.10%
Epoch 2/10 | Loss: 142.1880 | Val Acc: 69.65%
Epoch 3/10 | Loss: 125.3557 | Val Acc: 73.05%
Epoch 4/10 | Loss: 106.0533 | Val Acc: 74.40%
Epoch 5/10 | Loss: 86.7683 | Val Acc: 74.90%
Epoch 6/10 | Loss: 65.9178 | Val Acc: 75.45%
Epoch 7/10 | Loss: 49.6651 | Val Acc: 74.75%
Epoch 8/10 | Loss: 35.3526 | Val Acc: 75.10%
Epoch 9/10 | Loss: 26.3396 | Val Acc: 74.85%
Epoch 10/10 | Loss: 20.8749 | Val Acc: 74.60%


In [19]:
model.eval()
correct = total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = (outputs > 0.5).squeeze().long()
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Final Validation Accuracy: {100 * correct / total:.2f}%")


Final Validation Accuracy: 92.62%


'd:\\ds\\DL\\My_practices'